<h1>Question 1</h1>

You're given a dataset of products, categories, and their prices.
 Your task is to rank products by price within each category and classify them as:

🟢 "Price Up" if the price is higher than the previous product in that category

🔴 "Price Down" if the price is lower

⚪ "Stable" if the price is the same or it's the first product in that category


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *


In [2]:
spark = SparkSession.builder.appName("question_1").getOrCreate()

25/04/22 22:51:00 WARN Utils: Your hostname, Yashs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.3.100 instead (on interface en0)
25/04/22 22:51:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 22:51:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/22 22:51:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = [
 ("ProductA", "Electronics", 1000),
 ("ProductB", "Electronics", 1200),
 ("ProductC", "Electronics", 1100),
 ("ProductD", "Clothing", 50),
 ("ProductE", "Clothing", 40),
 ("ProductF", "Clothing", 50),
]

In [4]:
column = ["product", "category", "price"]

In [5]:
df = spark.createDataFrame(data,column)

In [6]:
window_spec = Window.partitionBy("category").orderBy("price")

In [7]:
df_with_lag = df.withColumn("previous_price",lag("price").over(window_spec))

In [8]:
df_with_lag.show()

+--------+-----------+-----+--------------+
| product|   category|price|previous_price|
+--------+-----------+-----+--------------+
|ProductE|   Clothing|   40|          NULL|
|ProductD|   Clothing|   50|            40|
|ProductF|   Clothing|   50|            50|
|ProductA|Electronics| 1000|          NULL|
|ProductC|Electronics| 1100|          1000|
|ProductB|Electronics| 1200|          1100|
+--------+-----------+-----+--------------+



In [9]:
final_rank_df = df_with_lag.withColumn("price_trend",
                                       when(col("previous_price").isNull(),"stable").
                                       when(col("price") > col("previous_price"),"price_up").
                                       when(col("price") < col("previous_price"),"price_up").
                                       otherwise("stable")
                                       )

In [10]:
final_rank_df.show()


+--------+-----------+-----+--------------+-----------+
| product|   category|price|previous_price|price_trend|
+--------+-----------+-----+--------------+-----------+
|ProductE|   Clothing|   40|          NULL|     stable|
|ProductD|   Clothing|   50|            40|   price_up|
|ProductF|   Clothing|   50|            50|     stable|
|ProductA|Electronics| 1000|          NULL|     stable|
|ProductC|Electronics| 1100|          1000|   price_up|
|ProductB|Electronics| 1200|          1100|   price_up|
+--------+-----------+-----+--------------+-----------+

